# Notebook 1: Data Preparation

**Objective:** Load the raw `Books.csv`, `Users.csv`, and `Ratings.csv` files, clean them, handle missing values, and export the cleaned DataFrames to the `artifacts/` directory for use in other notebooks and the app.

In [1]:
import pandas as pd
import numpy as np
import os

# Define file paths
BOOKS_PATH = '../data/Books.csv'
USERS_PATH = '../data/Users.csv'
RATINGS_PATH = '../data/Ratings.csv'

# Define output directory
ARTIFACTS_DIR = '../artifacts'

# Create artifacts directory if it doesn't exist
if not os.path.exists(ARTIFACTS_DIR):
    os.makedirs(ARTIFACTS_DIR)

## 1. Load and Clean `Books.csv`

In [2]:
# Load books data
# Specify encoding due to potential special characters
try:
    books_df = pd.read_csv(BOOKS_PATH, sep=',', encoding='latin-1', on_bad_lines='skip', low_memory=False)
except Exception as e:
    print(f"Error loading Books.csv: {e}")
    # Try a different encoding if latin-1 fails
    books_df = pd.read_csv(BOOKS_PATH, sep=',', encoding='ISO-8859-1', on_bad_lines='skip', low_memory=False)

print(f"Original shape of Books: {books_df.shape}")
books_df.head()

Original shape of Books: (271360, 8)


,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L
0,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...
1,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...
2,0060973129,Decision in Normandy,Carlo D'Este,1991,HarperPerennial,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...
3,0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,1999,Farrar Straus Giroux,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...
4,0393045218,The Mummies of Urumchi,E. J. W. Barber,1999,W. W. Norton &amp; Company,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...


In [3]:
# Check for null values
books_df.isnull().sum()

ISBN                   0
Book-Title             0
Book-Author            2
Year-Of-Publication    0
Publisher              2
Image-URL-S            0
Image-URL-M            0
Image-URL-L            3
dtype: int64

In [4]:
# Clean column names (lowercase and replace hyphens)
books_df.columns = books_df.columns.str.lower().str.replace('-', '_')
books_df.columns

Index(['isbn', 'book_title', 'book_author', 'year_of_publication', 'publisher',
       'image_url_s', 'image_url_m', 'image_url_l'],
      dtype='object')

In [5]:
# Clean 'year_of_publication'
# This column has some non-numeric values (e.g., 'DK Publishing Inc', 'Gallimard')
books_df['year_of_publication'] = pd.to_numeric(books_df['year_of_publication'], errors='coerce')

# Replace 0s and very high/low years with NaN
books_df.loc[(books_df['year_of_publication'] > 2024) | (books_df['year_of_publication'] < 1800), 'year_of_publication'] = np.nan

# Impute NaN values with the median year
median_year = books_df['year_of_publication'].median()
books_df['year_of_publication'].fillna(median_year, inplace=True)

# Convert to integer
books_df['year_of_publication'] = books_df['year_of_publication'].astype(int)

print(f"Median year used for imputation: {median_year}")
print("Cleaned 'year_of_publication' info:")
books_df['year_of_publication'].describe()

Median year used for imputation: 1996.0
Cleaned 'year_of_publication' info:


C:\Users\sambi\AppData\Local\Temp\ipykernel_19876\3878916613.py:10: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  books_df['year_of_publication'].fillna(median_year, inplace=True)


count    271360.000000
mean       1993.731983
std           8.083916
min        1806.000000
25%        1989.000000
50%        1996.000000
75%        2000.000000
max        2024.000000
Name: year_of_publication, dtype: float64

In [6]:
# Impute missing 'book_author' and 'publisher' with 'Unknown'
books_df['book_author'].fillna('Unknown', inplace=True)
books_df['publisher'].fillna('Unknown', inplace=True)

# Check nulls again
books_df.isnull().sum()

C:\Users\sambi\AppData\Local\Temp\ipykernel_19876\3221317863.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  books_df['book_author'].fillna('Unknown', inplace=True)
C:\Users\sambi\AppData\Local\Temp\ipykernel_19876\3221317863.py:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.



isbn                   0
book_title             0
book_author            0
year_of_publication    0
publisher              0
image_url_s            0
image_url_m            0
image_url_l            3
dtype: int64

## 2. Load and Clean `Users.csv`

In [7]:
# Load users data
try:
    users_df = pd.read_csv(USERS_PATH, sep=',', encoding='latin-1', on_bad_lines='skip')
except Exception as e:
    users_df = pd.read_csv(USERS_PATH, sep=',', encoding='ISO-8859-1', on_bad_lines='skip')

print(f"Original shape of Users: {users_df.shape}")
users_df.head()

Original shape of Users: (278858, 3)


,User-ID,Location,Age
0,1,"nyc, new york, usa",NaN
1,2,"stockton, california, usa",18.0
2,3,"moscow, yukon territory, russia",NaN
3,4,"porto, v.n.gaia, portugal",17.0
4,5,"farnborough, hants, united kingdom",NaN


In [8]:
# Clean column names
users_df.columns = users_df.columns.str.lower().str.replace('-', '_')
users_df.columns

Index(['user_id', 'location', 'age'], dtype='object')

In [9]:
# Check 'age' column
users_df['age'].describe()

count    168096.000000
mean         34.751434
std          14.428097
min           0.000000
25%          24.000000
50%          32.000000
75%          44.000000
max         244.000000
Name: age, dtype: float64

In [10]:
# Handle outliers and NaNs in 'age'
# Ages above 100 and below 5 seem unlikely
users_df.loc[(users_df['age'] > 100) | (users_df['age'] < 5), 'age'] = np.nan

# Impute with median age
median_age = users_df['age'].median()
users_df['age'].fillna(median_age, inplace=True)
users_df['age'] = users_df['age'].astype(int)

print(f"Median age used for imputation: {median_age}")
users_df['age'].describe()

Median age used for imputation: 32.0


C:\Users\sambi\AppData\Local\Temp\ipykernel_19876\1351437419.py:7: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  users_df['age'].fillna(median_age, inplace=True)


count    278858.000000
mean         33.643385
std          10.630979
min           5.000000
25%          29.000000
50%          32.000000
75%          35.000000
max         100.000000
Name: age, dtype: float64

In [11]:
# Clean 'location' column - split into city, state, country
# This can be complex, for now, we'll just fill NaNs and extract country
users_df['location'].fillna('unknown, unknown, unknown', inplace=True)

def extract_country(location_str):
    try:
        return location_str.split(',')[-1].strip()
    except:
        return 'unknown'

users_df['country'] = users_df['location'].apply(extract_country)
users_df.head()

C:\Users\sambi\AppData\Local\Temp\ipykernel_19876\1115361902.py:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  users_df['location'].fillna('unknown, unknown, unknown', inplace=True)


,user_id,location,age,country
0,1,"nyc, new york, usa",32,usa
1,2,"stockton, california, usa",18,usa
2,3,"moscow, yukon territory, russia",32,russia
3,4,"porto, v.n.gaia, portugal",17,portugal
4,5,"farnborough, hants, united kingdom",32,united kingdom


## 3. Load and Clean `Ratings.csv`

In [12]:
# Load ratings data
try:
    ratings_df = pd.read_csv(RATINGS_PATH, sep=',', encoding='latin-1', on_bad_lines='skip')
except Exception as e:
    ratings_df = pd.read_csv(RATINGS_PATH, sep=',', encoding='ISO-8859-1', on_bad_lines='skip')

print(f"Original shape of Ratings: {ratings_df.shape}")
ratings_df.head()

Original shape of Ratings: (1149780, 3)


,User-ID,ISBN,Book-Rating
0,276725,034545104X,0
1,276726,0155061224,5
2,276727,0446520802,0
3,276729,052165615X,3
4,276729,0521795028,6


In [13]:
# Clean column names
ratings_df.columns = ratings_df.columns.str.lower().str.replace('-', '_')
ratings_df.columns

Index(['user_id', 'isbn', 'book_rating'], dtype='object')

In [14]:
# Check for nulls
ratings_df.isnull().sum()

user_id        0
isbn           0
book_rating    0
dtype: int64

In [15]:
# Ratings dataset looks clean. No missing values.
# Check data types
ratings_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1149780 entries, 0 to 1149779
Data columns (total 3 columns):
 #   Column       Non-Null Count    Dtype 
---  ------       --------------    ----- 
 0   user_id      1149780 non-null  int64 
 1   isbn         1149780 non-null  object
 2   book_rating  1149780 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 26.3+ MB


## 4. Export Cleaned Data

Save the cleaned DataFrames as pickle files for easy loading in other notebooks.

In [16]:
books_df.to_pickle(os.path.join(ARTIFACTS_DIR, 'cleaned_books.pkl'))
users_df.to_pickle(os.path.join(ARTIFACTS_DIR, 'cleaned_users.pkl'))
ratings_df.to_pickle(os.path.join(ARTIFACTS_DIR, 'cleaned_ratings.pkl'))

print(f"Cleaned data saved to {ARTIFACTS_DIR}")
print(f"Books shape: {books_df.shape}")
print(f"Users shape: {users_df.shape}")
print(f"Ratings shape: {ratings_df.shape}")

Cleaned data saved to ../artifacts
Books shape: (271360, 8)
Users shape: (278858, 4)
Ratings shape: (1149780, 3)
